# Analyze terrain-related controls on snow depth

In [ ]:
import os
import glob
import xarray as xr
import rioxarray as rxr
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
import numpy as np
from topo_descriptors import topo
import xrspatial
import pandas as pd

In [ ]:
### Define paths in directory ###
refdem_fn = '/Volumes/LaCie/raineyaberle/Research/PhD/SkySat-Stereo/study-sites/MCS/refdem/MCS_REFDEM_WGS84.tif'
sd_dir = '/Volumes/LaCie/raineyaberle/Research/PhD/SkySat-Stereo/snow_depth_maps/'
sd_fns = sorted(glob.glob(os.path.join(sd_dir, 'MCS*.tif')))
out_dir = os.path.join(sd_dir, 'terrain_analysis')

sd_fns

In [ ]:
# Load reference DEM
refdem = xr.open_dataset(refdem_fn)
refdem = refdem.rename({'band_data': 'elevation'})
# coarsen to 5 m
dx = refdem.x.data[1] - refdem.x.data[0]
dx_new = 10
refdem = (refdem
          .coarsen(x=int(dx_new/dx), boundary='trim').mean()
          .coarsen(y=int(dx_new/dx), boundary='trim').mean())
# Calculate terrain parameters
refdem['slope'] = xrspatial.slope(refdem.elevation.squeeze())
refdem['curvature'] = xrspatial.curvature(refdem.elevation.squeeze())
refdem['aspect'] = xrspatial.aspect(refdem.elevation.squeeze())
refdem['elevation'] = refdem['elevation'].squeeze()

# Plot
plt.rcParams.update({'font.sans-serif': 'Arial', 'font.size': 12})
fig, ax = plt.subplots(2,2, figsize=(10,8))
ax = ax.flatten()
cmaps = [plt.cm.terrain, plt.cm.Reds, plt.cm.coolwarm_r, plt.cm.twilight]
for i, (cmap, column) in enumerate(zip(cmaps, ['elevation', 'slope', 'curvature', 'aspect'])):
    if i==2:
        ax_im = ax[i].imshow(refdem[column].data, cmap=cmap, clim=(-5, 5),
                         extent=(np.min(refdem.x)/1e3, np.max(refdem.x)/1e3, 
                                 np.min(refdem.y)/1e3, np.max(refdem.y)/1e3))
    else:
        ax_im = ax[i].imshow(refdem[column].data, cmap=cmap,
                             extent=(np.min(refdem.x)/1e3, np.max(refdem.x)/1e3, 
                                     np.min(refdem.y)/1e3, np.max(refdem.y)/1e3))
    fig.colorbar(ax_im, ax=ax[i], shrink=0.8)
    ax[i].set_title(column)
plt.show()

# Save figure
# fig_fn = os.path.join(out_dir, 'terrain_params.png')
# fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
# print('Figure saved to file:', fig_fn)

In [ ]:
## Train model and predict snow depths using terrain parameters
Xcols = ['elevation', 'slope', 'aspect', 'curvature']
ycols = ['snow_depth']

for sd_fn in [sd_fns[-1]]:
    # Load snow depths, interpolate to reference DEM coordinates
    sd = xr.open_dataset(sd_fn)
    sd = sd.sel(x=refdem.x, y=refdem.y, method='nearest')

    # identify pixels with real values for reshaping later
    ix = [np.where((np.isfinite(refdem[col].data) 
                    & ~np.isnan(refdem[col].data) 
                    & ~np.isnan(sd.band_data.data[0])), True, False)
          for col in Xcols]
    ireal = np.full(np.shape(refdem[Xcols[0]].data), True)
    for ixx in ix:
        ireal = ireal & ixx

    # Construct training data
    training_data_df = pd.DataFrame()
    for col in Xcols:
        training_data_df[col] = np.ravel(refdem[col].data)
    training_data_df['snow_depth'] = np.ravel(sd.band_data.data)
    training_data_df.dropna(inplace=True)
    X = training_data_df[Xcols]
    y = training_data_df[ycols]

    # Train linear model
    # svr = SVR().fit(X,y)
    lr = LinearRegression().fit(X,y)
    # score = lr.score(X,y)
    ypred = lr.predict(X)

    # Reshape predictions into image
    sd_pred = np.full(np.shape(refdem[Xcols[0]].data), np.nan)
    sd_pred[ireal] = np.ravel(ypred)

    # Plot results
    fig, ax = plt.subplots(1, 3, figsize=(14,6))
    # Observed
    ax_im = ax[0].imshow(sd.band_data.data[0], cmap='Blues', clim=(0,2.5),
                         extent=(np.min(sd.x)/1e3, np.max(sd.y)/1e3, 
                                 np.min(sd.x)/1e3, np.max(sd.y)/1e3))
    ax[0].set_title('Observed')
    ax[0].set_xlabel('Easting [km]')
    ax[0].set_ylabel('Northing [km]')
    fig.colorbar(ax_im, ax=ax[0], orientation='horizontal', label='Snow depth [m]')
    # Predicted
    ax_im = ax[1].imshow(sd_pred, cmap='Blues', clim=(0,2.5),
                         extent=(np.min(sd.x)/1e3, np.max(sd.y)/1e3, 
                                 np.min(sd.x)/1e3, np.max(sd.y)/1e3))
    ax[1].set_title('Predicted')
    ax[1].set_xlabel('Easting [km]')
    fig.colorbar(ax_im, ax=ax[1], orientation='horizontal', label='Snow depth [m]')
    # Difference
    ax_im = ax[2].imshow(sd_pred - sd.band_data.data[0], cmap='coolwarm_r', clim=(-1,1),
                         extent=(np.min(sd.x)/1e3, np.max(sd.y)/1e3, 
                                 np.min(sd.x)/1e3, np.max(sd.y)/1e3))
    fig.colorbar(ax_im, ax=ax[2], orientation='horizontal', label='Difference [m]')
    ax[2].set_title('Difference')

    fig.suptitle(os.path.basename(sd_fn))
    plt.show()

    # Save figure
    # fig_fn = os.path.join(out_dir, f'terrain_model_{os.path.basename(sd_fn)}')
    # fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
    # print('Figure saved to file:', fig_fn)

In [ ]:
svr = SVR().fit(X[0::10], y[0::10])

ypred = svr.predict(X)
sd_pred = np.full(np.shape(refdem[Xcols[0]].data), np.nan)
sd_pred[ireal] = np.ravel(ypred)

# Plot results
fig, ax = plt.subplots(1, 3, figsize=(14,6))
# Observed
ax_im = ax[0].imshow(sd.band_data.data[0], cmap='Blues', clim=(0,2.5),
                     extent=(np.min(sd.x)/1e3, np.max(sd.y)/1e3, 
                             np.min(sd.x)/1e3, np.max(sd.y)/1e3))
ax[0].set_title('Observed')
ax[0].set_xlabel('Easting [km]')
ax[0].set_ylabel('Northing [km]')
fig.colorbar(ax_im, ax=ax[0], orientation='horizontal', label='Snow depth [m]')
# Predicted
ax_im = ax[1].imshow(sd_pred, cmap='Blues', clim=(0,2.5),
                     extent=(np.min(sd.x)/1e3, np.max(sd.y)/1e3, 
                             np.min(sd.x)/1e3, np.max(sd.y)/1e3))
ax[1].set_title('Predicted')
ax[1].set_xlabel('Easting [km]')
fig.colorbar(ax_im, ax=ax[1], orientation='horizontal', label='Snow depth [m]')
# Difference
ax_im = ax[2].imshow(sd_pred - sd.band_data.data[0], cmap='coolwarm_r', clim=(-1,1),
                     extent=(np.min(sd.x)/1e3, np.max(sd.y)/1e3, 
                             np.min(sd.x)/1e3, np.max(sd.y)/1e3))
fig.colorbar(ax_im, ax=ax[2], orientation='horizontal', label='Difference [m]')
ax[2].set_title('Difference')

fig.suptitle(os.path.basename(sd_fn))
plt.show()